Attempting to vectorise addition of food at random locations.

In [1]:
import torch
import numpy as np

In [2]:
A = torch.Tensor([
    [0, 0, 1, 1],
    [0, 0, 1, 2],
    [0, 0, 2, 1],
    [0, 0, 2, 2],
    [1, 0, 1, 1],
    [1, 0, 1, 2],
    [1, 0, 2, 1],
    [1, 0, 2, 2],
    [2, 0, 1, 1],
    [2, 0, 1, 2],
    [2, 0, 2, 1],
    [2, 0, 2, 2],
    [3, 0, 1, 1],
    [3, 0, 1, 2],
    [3, 0, 2, 1],
    [3, 0, 2, 2],
])
A

tensor([[0., 0., 1., 1.],
        [0., 0., 1., 2.],
        [0., 0., 2., 1.],
        [0., 0., 2., 2.],
        [1., 0., 1., 1.],
        [1., 0., 1., 2.],
        [1., 0., 2., 1.],
        [1., 0., 2., 2.],
        [2., 0., 1., 1.],
        [2., 0., 1., 2.],
        [2., 0., 2., 1.],
        [2., 0., 2., 2.],
        [3., 0., 1., 1.],
        [3., 0., 1., 2.],
        [3., 0., 2., 1.],
        [3., 0., 2., 2.]])

In [3]:
torch.unique(A, dim=0)

tensor([[0., 0., 1., 1.],
        [0., 0., 1., 2.],
        [0., 0., 2., 1.],
        [0., 0., 2., 2.],
        [1., 0., 1., 1.],
        [1., 0., 1., 2.],
        [1., 0., 2., 1.],
        [1., 0., 2., 2.],
        [2., 0., 1., 1.],
        [2., 0., 1., 2.],
        [2., 0., 2., 1.],
        [2., 0., 2., 2.],
        [3., 0., 1., 1.],
        [3., 0., 1., 2.],
        [3., 0., 2., 1.],
        [3., 0., 2., 2.]])

In [4]:
print(A[:, :1])
print(A[:, 1:])

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [2.],
        [2.],
        [2.],
        [2.],
        [3.],
        [3.],
        [3.],
        [3.]])
tensor([[0., 1., 1.],
        [0., 1., 2.],
        [0., 2., 1.],
        [0., 2., 2.],
        [0., 1., 1.],
        [0., 1., 2.],
        [0., 2., 1.],
        [0., 2., 2.],
        [0., 1., 1.],
        [0., 1., 2.],
        [0., 2., 1.],
        [0., 2., 2.],
        [0., 1., 1.],
        [0., 1., 2.],
        [0., 2., 1.],
        [0., 2., 2.]])


In [5]:
mask = torch.zeros(len(A))
mask

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [6]:
aux = A[:, :1]

In [7]:
mask[:1] = 1
mask[1:] = (aux[1:] != aux[:-1]).squeeze()
mask

tensor([1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.])

In [8]:
A[mask.byte()]

tensor([[0., 0., 1., 1.],
        [1., 0., 1., 1.],
        [2., 0., 1., 1.],
        [3., 0., 1., 1.]])

Not bad let's add some randomisation as that will always pick the first row for each image.

In [34]:
def unique1d(tensor, return_index=False):
    """Port of np.unique to PyTorch with the full `return_index` functionality"""
    assert len(tensor.shape) == 1
    
    optional_indices = return_index

    if optional_indices:
        perm = tensor.argsort()
        aux = tensor[perm]
    else:
        tensor.sort_()#[0]
        aux = tensor
        
    mask = torch.zeros(aux.shape)
    mask[:1] = 1
    mask[1:] = aux[1:] != aux[:-1]
    
    ret = (aux[mask.byte()],)
    if return_index:
        ret += (perm[mask.byte()],)
        
    return ret
    
    

In [35]:
unique1d(A[:, 0], return_index=True)

(tensor([0., 1., 2., 3.]), tensor([ 0,  6,  8, 12]))

In [36]:
A[torch.randperm(len(A))]

tensor([[2., 0., 2., 1.],
        [3., 0., 1., 2.],
        [1., 0., 2., 2.],
        [2., 0., 1., 1.],
        [3., 0., 2., 1.],
        [1., 0., 1., 2.],
        [0., 0., 1., 1.],
        [0., 0., 2., 2.],
        [3., 0., 1., 1.],
        [1., 0., 1., 1.],
        [0., 0., 2., 1.],
        [3., 0., 2., 2.],
        [1., 0., 2., 1.],
        [2., 0., 2., 2.],
        [0., 0., 1., 2.],
        [2., 0., 1., 2.]])

In [37]:
unique1d(A[torch.randperm(len(A))][:, 0], return_index=True)

(tensor([0., 1., 2., 3.]), tensor([ 2,  0, 10, 11]))

In [48]:
def get_unique_by_column(tensor, column):
    assert len(tensor.shape) == 2
    
    tensor = tensor[torch.randperm(len(tensor))]
    
    uniq = tensor[:, column]
    
    indices = unique1d(uniq, return_index=True)[1]
    
    return tensor[indices.long()]
    

get_unique_by_column(A, 0)

tensor([[0., 0., 1., 1.],
        [1., 0., 1., 1.],
        [2., 0., 2., 2.],
        [3., 0., 2., 2.]])